# 1. 데이터 불러오기

In [65]:
import os
os.chdir('C:/Users/hu612/Desktop/School/수DA쟁이/Project/개인과제/data')

In [66]:
import pandas as pd
import numpy as np

train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

# 2. TRAIN TEST 나눈 LSTM

In [6]:
import tensorflow as tf
X_train = train[(train['id']>= 0) & (train['id']<=2499)]
X_test = train[train['id'] >= 2500]
X_train = tf.reshape(np.array(X_train.iloc[:,2:]),[-1, 600, 6])
X_test = tf.reshape(np.array(X_test.iloc[:,2:]),[-1, 600, 6])

from tensorflow.keras.utils import to_categorical
y =  train_labels['label']
slice_arr = slice(0,2500,1)
y_train = y[slice_arr]

slice_arr2 = slice(2500,3125,1)
y_test = y[slice_arr2]

y_train_encoding = tf.keras.utils.to_categorical(y_train)
y_test_encoding = tf.keras.utils.to_categorical(y_test)

In [7]:
# op= adam,loss mse 했을때 정확도 0 나옴
from keras.models import Sequential
from keras.layers import LSTM,Dense

np.random.seed(0)
model = Sequential()
model.add(LSTM(32, input_shape=(600,6)))
model.add(Dense(128, activation='relu'))
model.add(Dense(61,activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4992      
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dense_1 (Dense)              (None, 61)                7869      
Total params: 17,085
Trainable params: 17,085
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(X_train,y_train_encoding
          , epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
16/16 [==============================] - 14s 626ms/step - loss: 3.8574 - accuracy: 0.2402 - val_loss: 3.3378 - val_accuracy: 0.4840
Epoch 2/30
16/16 [==============================] - 8s 493ms/step - loss: 3.1679 - accuracy: 0.4853 - val_loss: 2.8870 - val_accuracy: 0.4860
Epoch 3/30
16/16 [==============================] - 8s 479ms/step - loss: 2.8512 - accuracy: 0.4757 - val_loss: 2.7706 - val_accuracy: 0.4860
Epoch 4/30
16/16 [==============================] - 8s 517ms/step - loss: 2.7729 - accuracy: 0.4687 - val_loss: 2.7425 - val_accuracy: 0.4860
Epoch 5/30
16/16 [==============================] - 8s 475ms/step - loss: 2.6752 - accuracy: 0.4783 - val_loss: 2.7171 - val_accuracy: 0.4860
Epoch 6/30
16/16 [==============================] - 7s 431ms/step - loss: 2.6602 - accuracy: 0.4716 - val_loss: 2.6943 - val_accuracy: 0.4860
Epoch 7/30
16/16 [==============================] - 8s 493ms/step - loss: 2.5704 - accuracy: 0.4824 - val_loss: 2.6706 - val_accuracy: 0.4860
Epoch

In [9]:
y_pred = model.predict(X_test)

In [10]:
y_pred = model.predict(X_test)
y_pred1 = list(y_pred)
for a in range(0,625,1) :
    for i in range(0,61,1) :
        if (i == np.argmax(y_pred1[a])) : 
            y_pred1[a][i] = 1
        else :
            y_pred1[a][i] = 0

In [11]:
print("테스트 세트의 정확도: {:.2f}".format(np.mean(y_pred == y_test_encoding)))

테스트 세트의 정확도: 0.98


In [12]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape

TensorShape([782, 600, 6])

In [13]:
prediction=model.predict(test_X)

In [14]:
submission.iloc[:,1:]=prediction

In [15]:
submission.to_csv('C:/Users/hu612/Desktop/School/수DA쟁이/Project/개인과제/data/LSTM/2.csv', index=False)

=>LSTM2 : 2.47035

# 3. 좀 더 깊이 있는 LSTM

In [16]:
import tensorflow as tf
X_train = train[(train['id']>= 0) & (train['id']<=2499)]
X_test = train[train['id'] >= 2500]
X_train = tf.reshape(np.array(X_train.iloc[:,2:]),[-1, 600, 6])
X_test = tf.reshape(np.array(X_test.iloc[:,2:]),[-1, 600, 6])

from tensorflow.keras.utils import to_categorical
y =  train_labels['label']
slice_arr = slice(0,2500,1)
y_train = y[slice_arr]

slice_arr2 = slice(2500,3125,1)
y_test = y[slice_arr2]

y_train_encoding = tf.keras.utils.to_categorical(y_train)
y_test_encoding = tf.keras.utils.to_categorical(y_test)

In [18]:
# op= adam,loss mse 했을때 정확도 0 나옴
from keras.models import Sequential
from keras.layers import LSTM,Dense, Flatten

np.random.seed(0)
model = Sequential()
model.add(LSTM(32, input_shape=(600,6), return_sequences=True))
model.add(Dense(128, activation='relu'))
model.add(Flatten())
model.add(Dense(61,activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 600, 32)           4992      
_________________________________________________________________
dense_4 (Dense)              (None, 600, 128)          4224      
_________________________________________________________________
flatten (Flatten)            (None, 76800)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 61)                4684861   
Total params: 4,694,077
Trainable params: 4,694,077
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 엄청난 과적합 이므로 Dropout을 추가 후 해보자
history = model.fit(X_train,y_train_encoding
          , epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
16/16 [==============================] - 16s 757ms/step - loss: 12.1253 - accuracy: 0.1961 - val_loss: 7.8498 - val_accuracy: 0.0380
Epoch 2/30
16/16 [==============================] - 11s 709ms/step - loss: 4.2115 - accuracy: 0.3785 - val_loss: 2.7166 - val_accuracy: 0.5400
Epoch 3/30
16/16 [==============================] - 11s 715ms/step - loss: 0.7149 - accuracy: 0.8330 - val_loss: 2.8549 - val_accuracy: 0.5300
Epoch 4/30
16/16 [==============================] - 11s 702ms/step - loss: 0.5849 - accuracy: 0.8342 - val_loss: 3.4094 - val_accuracy: 0.5200
Epoch 5/30
16/16 [==============================] - 11s 692ms/step - loss: 0.2840 - accuracy: 0.9087 - val_loss: 4.5546 - val_accuracy: 0.5220
Epoch 6/30
16/16 [==============================] - 11s 687ms/step - loss: 0.2380 - accuracy: 0.9434 - val_loss: 2.5267 - val_accuracy: 0.5620
Epoch 7/30
16/16 [==============================] - 11s 716ms/step - loss: 0.0277 - accuracy: 0.9940 - val_loss: 2.8638 - val_accuracy: 0.556

KeyboardInterrupt: 

In [23]:
# op= adam,loss mse 했을때 정확도 0 나옴
from keras.models import Sequential
from keras.layers import LSTM,Dense, Flatten,Dropout

np.random.seed(0)
model = Sequential()
model.add(LSTM(32, input_shape=(600,6), return_sequences=True))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(61,activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 600, 32)           4992      
_________________________________________________________________
dropout_4 (Dropout)          (None, 600, 32)           0         
_________________________________________________________________
dense_10 (Dense)             (None, 600, 128)          4224      
_________________________________________________________________
dropout_5 (Dropout)          (None, 600, 128)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 76800)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 61)                4684861   
Total params: 4,694,077
Trainable params: 4,694,077
Non-trainable params: 0
____________________________________________

In [24]:
history = model.fit(X_train,y_train_encoding
          , epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
16/16 [==============================] - 19s 950ms/step - loss: 10.9421 - accuracy: 0.2185 - val_loss: 4.3121 - val_accuracy: 0.4240
Epoch 2/30
16/16 [==============================] - 15s 914ms/step - loss: 2.5247 - accuracy: 0.5520 - val_loss: 2.9495 - val_accuracy: 0.4400
Epoch 3/30
16/16 [==============================] - 14s 909ms/step - loss: 1.0409 - accuracy: 0.7166 - val_loss: 2.7615 - val_accuracy: 0.4400
Epoch 4/30
16/16 [==============================] - 14s 906ms/step - loss: 0.5299 - accuracy: 0.8673 - val_loss: 2.4616 - val_accuracy: 0.5000
Epoch 5/30
16/16 [==============================] - 14s 901ms/step - loss: 0.1843 - accuracy: 0.9602 - val_loss: 2.5052 - val_accuracy: 0.5380
Epoch 6/30
16/16 [==============================] - 14s 910ms/step - loss: 0.2102 - accuracy: 0.9557 - val_loss: 2.8423 - val_accuracy: 0.5240
Epoch 7/30
16/16 [==============================] - 14s 897ms/step - loss: 0.1148 - accuracy: 0.9689 - val_loss: 2.5559 - val_accuracy: 0.566

In [25]:
y_pred = model.predict(X_test)

In [26]:
y_pred

array([[3.8271796e-07, 2.9343821e-06, 5.4819588e-10, ..., 2.2442133e-08,
        1.4729980e-07, 9.8925526e-08],
       [1.2678292e-08, 2.2346822e-09, 7.5998834e-08, ..., 2.3385178e-08,
        7.1340085e-09, 1.2896312e-11],
       [3.8430630e-06, 4.2264305e-06, 9.0953101e-10, ..., 9.4389253e-07,
        1.2833038e-07, 5.2714982e-04],
       ...,
       [1.7837614e-12, 9.9386081e-16, 4.5308687e-13, ..., 3.7514176e-17,
        5.3825750e-12, 5.9461320e-14],
       [3.3270480e-05, 3.8214457e-10, 3.7643771e-05, ..., 6.8480822e-06,
        2.0725854e-05, 3.3739809e-08],
       [2.1589921e-09, 7.8185747e-10, 8.4737251e-03, ..., 1.8714115e-04,
        2.5855345e-04, 7.7055522e-11]], dtype=float32)

In [29]:
y_test_encoding

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [30]:
y_pred = model.predict(X_test)
y_pred1 = list(y_pred)
for a in range(0,625,1) :
    for i in range(0,61,1) :
        if (i == np.argmax(y_pred1[a])) : 
            y_pred1[a][i] = 1
        else :
            y_pred1[a][i] = 0

In [31]:
print("테스트 세트의 정확도: {:.2f}".format(np.mean(y_pred == y_test_encoding)))

테스트 세트의 정확도: 0.99


In [32]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape

TensorShape([782, 600, 6])

In [33]:
prediction=model.predict(test_X)

In [34]:
submission.iloc[:,1:]=prediction

In [35]:
submission.to_csv('C:/Users/hu612/Desktop/School/수DA쟁이/Project/개인과제/data/LSTM/LSTM3.csv', index=False)

LSTM3 : 4.3238

# 4. BILSTM

In [68]:
import tensorflow as tf
X_train = train[(train['id']>= 0) & (train['id']<=2499)]
X_test = train[train['id'] >= 2500]
X_train = tf.reshape(np.array(X_train.iloc[:,2:]),[-1, 600, 6])
X_test = tf.reshape(np.array(X_test.iloc[:,2:]),[-1, 600, 6])

from tensorflow.keras.utils import to_categorical
y =  train_labels['label']
slice_arr = slice(0,2500,1)
y_train = y[slice_arr]

slice_arr2 = slice(2500,3125,1)
y_test = y[slice_arr2]

y_train_encoding = tf.keras.utils.to_categorical(y_train)
y_test_encoding = tf.keras.utils.to_categorical(y_test)

In [69]:
X_train.shape

TensorShape([2500, 600, 6])

In [67]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

np.random.seed(0)
model = Sequential()
model.add(LSTM(32,input_shape=(600,6), return_sequences = True))
model.add(Bidirectional(LSTM(32)))#,input_shape=(600,6))))
model.add(Dense(128, activation='relu'))
model.add(Flatten())
model.add(Dense(61,activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 600, 32)           4992      
_________________________________________________________________
bidirectional_9 (Bidirection (None, 64)                16640     
_________________________________________________________________
dense_28 (Dense)             (None, 128)               8320      
_________________________________________________________________
flatten_12 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 61)                7869      
Total params: 37,821
Trainable params: 37,821
Non-trainable params: 0
_________________________________________________________________


In [71]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [72]:
history = model.fit(X_train,y_train_encoding
          , epochs=30, batch_size=128,callbacks=[es,mc], validation_split=0.2)

Epoch 1/30
16/16 [==============================] - 54s 3s/step - loss: 3.7101 - accuracy: 0.3070 - val_loss: 2.7920 - val_accuracy: 0.4860
Epoch 2/30
16/16 [==============================] - 42s 3s/step - loss: 2.7517 - accuracy: 0.4861 - val_loss: 2.7555 - val_accuracy: 0.4860
Epoch 3/30
16/16 [==============================] - 38s 2s/step - loss: 2.6723 - accuracy: 0.4881 - val_loss: 2.7129 - val_accuracy: 0.4860
Epoch 4/30
16/16 [==============================] - 39s 2s/step - loss: 2.6054 - accuracy: 0.4930 - val_loss: 2.7093 - val_accuracy: 0.4860
Epoch 5/30
16/16 [==============================] - 41s 3s/step - loss: 2.5949 - accuracy: 0.4865 - val_loss: 2.6604 - val_accuracy: 0.4880
Epoch 6/30
16/16 [==============================] - 39s 2s/step - loss: 2.4942 - accuracy: 0.4937 - val_loss: 2.6263 - val_accuracy: 0.4880
Epoch 7/30
16/16 [==============================] - 40s 2s/step - loss: 2.4569 - accuracy: 0.4898 - val_loss: 2.5889 - val_accuracy: 0.4940
Epoch 8/30
16/16 [==

In [64]:
from tensorflow.keras.models import load_model
print("테스트 정확도: %.4f" % model.evaluate(X_test, y_test_encoding)[1])

20/20 [==============================] - 3s 152ms/step - loss: 2.0583 - accuracy: 0.5360
테스트 정확도: 0.5360
